In [ ]:
import os
import math
import wandb

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary

In [2]:
config = {
    "data": '/kaggle/input/mel-spectrogram-for-gtzn-dataset/melspectrograms',
    "arch": 'ECASCNN',
    "workers": 4,
    "epochs": 100,
    "start_epoch": 0,
    "batch_size": 64,
    "lr": 0.1,
    "momentum": 0.9,
    "weight_decay": 1e-4,
    "resume": False,
    "evaluate": False,
    "checkpoint_dir": "/kaggle/working/checkpoints/"
}

In [ ]:
class eca_layer(nn.Module):
    def __init__(self, channel, k_size=3):
        super(eca_layer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size - 1) // 2, bias=False) 
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.avg_pool(x)

        y = self.conv(y.squeeze(-1).transpose(-1, -2)).transpose(-1, -2).unsqueeze(-1)

        y = self.sigmoid(y)

        return x * y.expand_as(x)
        


In [ ]:
class CBR(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(CBR, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,
                              stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

In [ ]:
class ECAS_CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(ECAS_CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            CBR(in_channels=1, out_channels=128, kernel_size=3, stride=1, padding=1),
            eca_layer(channel=128),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=(0, 1))
        )
        
        self.layer2 = nn.Sequential(
            CBR(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            eca_layer(channel=128),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer3 = nn.Sequential(
            CBR(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            eca_layer(channel=256),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=(2 - 1) // 2)
        )
        
        self.fc_layers = nn.Sequential(
            nn.Flatten(),  
            nn.Linear(4096, 128), 
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(128, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        out1 = self.layer1(x)
        out2 = self.layer2(out1)
        out3 = self.layer3(out2)
        output = self.fc_layers(out3)
        return output

In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", DEVICE)

Device:  cuda


In [ ]:
model = ECAS_CNN(10).to(DEVICE)

In [6]:
model = model.to(DEVICE)

In [7]:
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.SGD(model.parameters(),lr=config['lr'], momentum=config['momentum'], weight_decay=config['weight_decay'])

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config['epochs'], eta_min=0.005)


In [8]:
def load_model(model, optimizer=None, scheduler=None):
    path = os.path.join(config['checkpoint_dir'], 'last.pth')
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    else:
        optimizer = None
    if scheduler is not None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    else:
        scheduler = None
    epoch = checkpoint['epoch']
    best_acc = checkpoint['best_acc']
    return model, optimizer, scheduler, epoch, best_acc

In [9]:
if config['resume']:
        load_model(model, optimizer, scheduler)

In [10]:
cudnn.benchmark = True

# Data loading code
traindir = os.path.join(config['data'], 'train/')
valdir = os.path.join(config['data'], 'validation/')
testdir = os.path.join(config['data'], 'test/')

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  
    transforms.Resize((130, 13)), 
    transforms.ToTensor(),
    ])
)

val_dataset = datasets.ImageFolder(
       valdir,
       transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  
        transforms.Resize((130, 13)),
        transforms.ToTensor(),
      ])
)

test_dataset = datasets.ImageFolder(
       testdir,
       transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  
        transforms.Resize((130, 13)),
        transforms.ToTensor(),
      ])
)

In [11]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=config['batch_size'], shuffle=True,
        num_workers=config['workers'], pin_memory=True
 )

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=config['batch_size'], shuffle=False,
    num_workers=config['workers'], pin_memory=True)

test_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=config['batch_size'], shuffle=False,
    num_workers=config['workers'], pin_memory=True)

In [12]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [13]:
def accuracy(output, target):
    with torch.no_grad():
        batch_size = target.size(0)
        
        _, pred = output.max(1)
        
        correct = pred.eq(target).float().sum()
       
        acc = correct.mul(100.0 / batch_size)
        
        return acc


In [14]:
def validate(val_loader, model, criterion):
    losses = AverageMeter()
    accuracy_meter = AverageMeter()

    model.eval()

    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
           
            
            input = input.to(DEVICE, non_blocking=True)
            target = target.to(DEVICE, non_blocking=True)

    
            output = model(input)
            loss = criterion(output, target)

        
            acc = accuracy(output, target)
            losses.update(loss.item(), input.size(0))
            accuracy_meter.update(acc, input.size(0))


    return losses.avg, accuracy_meter.avg


In [15]:
def save_model(model, optimizer, scheduler, best_acc, epoch):
    path = os.path.join(config['checkpoint_dir'], 'last.pth')
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         'best_acc'                 : best_acc,
         'epoch'                    : epoch},
         path)

In [16]:
def train(train_loader, model, criterion, optimizer, epoch):
    losses = AverageMeter()
    accuracy_meter = AverageMeter()

    model.train()

    for i, (input, target) in enumerate(train_loader):
      
        input = input.to(DEVICE, non_blocking=True)
        target = target.to(DEVICE, non_blocking=True)


        output = model(input)
        loss = criterion(output, target)

    
        acc = accuracy(output, target)
        losses.update(loss.item(), input.size(0))
        accuracy_meter.update(acc, input.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    

    return losses.avg, accuracy_meter.avg


In [17]:
checkpoint_dir = config['checkpoint_dir']

os.makedirs(checkpoint_dir, exist_ok=True)

In [ ]:
if config['evaluate']:
        test_loss, test_acc = validate(test_loader, model, criterion)
        print("\tTest Acc {:.04f}%\tTest Loss {:.04f}".format(test_acc, test_loss))
else:
    best_acc = 0
    for epoch in range(config['start_epoch'], config['epochs']):

        curr_lr = float(optimizer.param_groups[0]['lr'])
        train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch)
        val_loss, val_acc = validate(val_loader, model, criterion)
      

        is_best = val_acc > best_acc
        best_acc = max(val_acc, best_acc)


        if(is_best):
            save_model(model, optimizer, scheduler, best_acc, epoch)
        
        scheduler.step()

        print(f"Epoch {epoch + 1}\n")
        print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc, train_loss, curr_lr))
        print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc, val_loss))

        

Epoch 1

	Train Acc 30.7933%	Train Loss 1.8728	 Learning Rate 0.1000000
	Val Acc 12.7127%	Val Loss 2.5594
Epoch 2

	Train Acc 33.6587%	Train Loss 1.7768	 Learning Rate 0.0999766
	Val Acc 16.3163%	Val Loss 2.9484
Epoch 3

	Train Acc 38.6136%	Train Loss 1.6455	 Learning Rate 0.0999063
	Val Acc 19.9199%	Val Loss 2.4950
Epoch 4

	Train Acc 40.2528%	Train Loss 1.5793	 Learning Rate 0.0997892
	Val Acc 29.2292%	Val Loss 1.7510
Epoch 5

	Train Acc 45.6707%	Train Loss 1.4771	 Learning Rate 0.0996254
	Val Acc 37.7377%	Val Loss 1.5951
Epoch 6

	Train Acc 48.5360%	Train Loss 1.4037	 Learning Rate 0.0994152
	Val Acc 27.4274%	Val Loss 2.0851
Epoch 7

	Train Acc 50.8759%	Train Loss 1.3646	 Learning Rate 0.0991586
	Val Acc 35.6356%	Val Loss 2.0887
Epoch 8

	Train Acc 55.5681%	Train Loss 1.2445	 Learning Rate 0.0988560
	Val Acc 42.2422%	Val Loss 1.8301
Epoch 9

	Train Acc 57.1822%	Train Loss 1.2034	 Learning Rate 0.0985077
	Val Acc 26.5265%	Val Loss 2.0260
Epoch 10

	Train Acc 61.5741%	Train Loss 1.118